# Project 5: Group Project
#### Author: Adam Pardo, Brandon Bergeron, Eric Bayless, Ramesh Babu

### 01 - Data Collection and Data Cleaning

Task:

Information: 

In [1]:
# import libraries here 
import pandas as pd
import os

In [5]:
# import Business.json
business_json_path = '../data/yelp_academic_dataset_business.json'
business = pd.read_json(business_json_path, lines=True)

In [6]:
# function to pull business dataset by city

def city_restaurants(business_json, city_name):
    is_open = business_json[business_json['is_open'] ==1]
    location = is_open[is_open['city'] == city_name]
    location = location.reset_index(drop=True)
    location = location.dropna(subset=['categories'])
    location_rest = location[location['categories'].str.contains('Restaurant')]
    location_rest = location_rest.reset_index(drop=True)
    location_rest_small = location_rest[(location_rest['review_count'] > 100) & (location_rest['review_count'] < 300)]
    
    #uncomment if you want csv of business in given city
    #location_rest_small.to_csv(f'./data/city_data/{city_name}_starter.csv', index = False)
    
    return location_rest_small

In [7]:
# calling city_restaurants function if you want to export csv of city restaurant data
city_restaurants(business, 'Las Vegas')

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
2,Yr_w9lakJrKMyEG_hI6zbA,Fat Moe's Pizza & Wings,"6125 W Tropicana Ave, Ste F",Las Vegas,NV,89103,36.099361,-115.226636,4.0,141,1,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Pizza, Salad, Burgers, Restaurants","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
3,AN0bWhisCf6LN9eHZ7DQ3w,Los Olivos Ristorante,3759 E Desert Inn Rd,Las Vegas,NV,89121,36.129178,-115.092483,5.0,222,1,"{'WiFi': 'u'free'', 'RestaurantsPriceRange2': ...","Restaurants, Italian","{'Monday': '0:0-0:0', 'Tuesday': '16:0-21:0', ..."
6,oUX2bYbqjqST-urKbOHG6w,Loftti Cafe,"7729 S Rainbow Blvd, Ste 9B",Las Vegas,NV,89139,36.047942,-115.244167,4.5,284,1,"{'OutdoorSeating': 'True', 'BusinessParking': ...","Sandwiches, Shaved Ice, Coffee & Tea, Desserts...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-3:0', 'W..."
7,FiW6w5nmhlUoJAyNofb4jg,Fruits and Roots,5020 Blue Diamond Rd,Las Vegas,NV,89139,36.032122,-115.210267,4.5,106,1,"{'BusinessAcceptsCreditCards': 'True', 'DogsAl...","Coffee & Tea, Food Stands, Food, Restaurants, ...",{'Monday': '0:0-0:0'}
16,T0NKethAB-FFR05EeZCzuA,Burger King,6780 N Durango Dr,Las Vegas,NV,89149,36.284274,-115.287331,1.5,127,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Burgers, Fast Food, Restaurants","{'Monday': '6:0-22:0', 'Tuesday': '6:0-22:0', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4365,HUKbH7r9TBJPri6LJbBKtw,Tacos Mexico,1800 Las Vegas Blvd S,Las Vegas,NV,89104,36.150032,-115.153714,2.5,168,1,"{'NoiseLevel': 'u'average'', 'RestaurantsTakeO...","Real Estate, Home Services, Real Estate Servic...","{'Monday': '9:0-9:0', 'Tuesday': '9:0-9:0', 'W..."
4370,CxZ_qwVlgd-DIrb_a3N8jw,Shish Kabob House,9255 S Eastern Ave,Las Vegas,NV,89123,36.021154,-115.119155,4.5,121,1,"{'Caters': 'True', 'OutdoorSeating': 'True', '...","Mediterranean, Armenian, Kebab, Middle Eastern...","{'Monday': '10:0-21:30', 'Tuesday': '10:0-21:3..."
4374,K19uBHydrYAsfX3CPnFp-w,Poukei,"3663 S Las Vegas Blvd, Ste 505",Las Vegas,NV,89109,36.109945,-115.170841,3.5,164,1,"{'BikeParking': 'False', 'WiFi': ''no'', 'Outd...","Asian Fusion, Japanese, Restaurants, Hawaiian","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
4375,P3xSYeiJFkbdBwpnEncPTg,Aroy Thai Kitchen,"4555 S Ft Apache Rd, Ste 112",Las Vegas,NV,89147,36.106477,-115.298066,4.5,182,1,"{'BusinessAcceptsCreditCards': 'True', 'Alcoho...","Restaurants, Thai","{'Monday': '0:0-0:0', 'Tuesday': '10:30-21:30'..."


In [42]:
# import Review.json
review_json_path = '../data/yelp_academic_dataset_review.json'
size = 500_000
rows = 1_000_000

review = pd.read_json(review_json_path, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

In [43]:
def business_reviews(review_json, business_json, city_name):
    chunk_list = []
    for chunk_review in review:
        chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
        chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
        chunk_merged = pd.merge(city_restaurants(business_json, city_name), chunk_review, on='business_id', how='inner')
        print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
        chunk_list.append(chunk_merged)
    df_reviews = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)
    
    #uncomment if you want csv of business in given city
    df_reviews.to_csv(f'./data/city_data/{city_name}_reviews.csv', index = False)
    return df_reviews

# https://towardsdatascience.com/converting-yelp-dataset-to-csv-using-pandas-2a4c8f03bd88

In [44]:
# function that joins city business information with yelp reviews
business_reviews(review, business, 'Las Vegas')

12155 out of 500,000 related reviews
13272 out of 500,000 related reviews


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,user_id,review_stars,text,date
0,Yr_w9lakJrKMyEG_hI6zbA,Fat Moe's Pizza & Wings,"6125 W Tropicana Ave, Ste F",Las Vegas,NV,89103,36.099361,-115.226636,4.0,141,1,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Pizza, Salad, Burgers, Restaurants","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",flaLsGwkBgWb_o_UVhXbTA,5,Love their food and their prices. The owners a...,2017-03-03 21:46:36
1,Yr_w9lakJrKMyEG_hI6zbA,Fat Moe's Pizza & Wings,"6125 W Tropicana Ave, Ste F",Las Vegas,NV,89103,36.099361,-115.226636,4.0,141,1,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Pizza, Salad, Burgers, Restaurants","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",yXhy8F43qDmSHuY9ubadtw,5,This place has definitely improved since the n...,2015-03-05 15:01:11
2,Yr_w9lakJrKMyEG_hI6zbA,Fat Moe's Pizza & Wings,"6125 W Tropicana Ave, Ste F",Las Vegas,NV,89103,36.099361,-115.226636,4.0,141,1,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Pizza, Salad, Burgers, Restaurants","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",Unhr6Ut9xQowzzohevNdxg,4,Ordered from here just awhile ago as I was cra...,2015-08-07 23:34:12
3,Yr_w9lakJrKMyEG_hI6zbA,Fat Moe's Pizza & Wings,"6125 W Tropicana Ave, Ste F",Las Vegas,NV,89103,36.099361,-115.226636,4.0,141,1,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Pizza, Salad, Burgers, Restaurants","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",LgkNGmjMijSiBlaWCfWtZw,5,Really yummy food that is made on the spot and...,2017-09-10 20:57:11
4,Yr_w9lakJrKMyEG_hI6zbA,Fat Moe's Pizza & Wings,"6125 W Tropicana Ave, Ste F",Las Vegas,NV,89103,36.099361,-115.226636,4.0,141,1,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Pizza, Salad, Burgers, Restaurants","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",FXB9CtInMisA1cET-Mhg5Q,5,We got here by accident since we were looking ...,2017-05-14 04:06:27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25422,8KEB1lcjKlIVSd6uA8JB9w,Cafe Rio,7040 S Durango Dr,Las Vegas,NV,89113,36.060258,-115.278690,2.5,272,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Mexican, Restaurants, Cafes","{'Monday': '10:30-22:0', 'Tuesday': '10:30-22:...",_XRwhR824qxBRYBCwWwVgQ,3,Arrived for lunch the place was just starting ...,2016-12-09 19:28:35
25423,8KEB1lcjKlIVSd6uA8JB9w,Cafe Rio,7040 S Durango Dr,Las Vegas,NV,89113,36.060258,-115.278690,2.5,272,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Mexican, Restaurants, Cafes","{'Monday': '10:30-22:0', 'Tuesday': '10:30-22:...",hyMzUeRlYYleebM-Os4JWw,1,"First off let me start by saying I feel ""stres...",2016-11-30 23:38:06
25424,8KEB1lcjKlIVSd6uA8JB9w,Cafe Rio,7040 S Durango Dr,Las Vegas,NV,89113,36.060258,-115.278690,2.5,272,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Mexican, Restaurants, Cafes","{'Monday': '10:30-22:0', 'Tuesday': '10:30-22:...",9xgTU7KxnPEwOKXkaPSY4A,1,This location has the absolute worst customer ...,2018-02-27 02:04:29
25425,8KEB1lcjKlIVSd6uA8JB9w,Cafe Rio,7040 S Durango Dr,Las Vegas,NV,89113,36.060258,-115.278690,2.5,272,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Mexican, Restaurants, Cafes","{'Monday': '10:30-22:0', 'Tuesday': '10:30-22:...",-a0lpGZIBodkwrrCLvRg8g,1,I rather go out of my way and go to the one on...,2018-01-28 03:41:42
